In [2]:
import os
import getpass

os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
)

In [10]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='The translation of "I love programming" from English to French is:\n\n"J\'adore programmer."', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-10-27T05:30:28.5855365Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4532562200, 'load_duration': 3997031700, 'prompt_eval_count': 35, 'prompt_eval_duration': 66433000, 'eval_count': 22, 'eval_duration': 464906000}, id='run-72028b5b-7bea-42c7-9849-aa90f0946ee9-0', usage_metadata={'input_tokens': 35, 'output_tokens': 22, 'total_tokens': 57})

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Indonesian",
        "input": "I love programming.",
    }
)

AIMessage(content='Suka sekali dengan pemrograman! (I like programming very much!)', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-10-27T05:30:32.0032316Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 527038500, 'load_duration': 59796800, 'prompt_eval_count': 30, 'prompt_eval_duration': 39168000, 'eval_count': 17, 'eval_duration': 424725000}, id='run-6d6dcf54-0335-4106-b2a7-f2deb989cf7f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 17, 'total_tokens': 47})

In [13]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': '["123 Fake St, Boston, MA", "234 Pretend Boulevard, Houston, TX"]',
   'user_id': 123},
  'id': '0adc6878-cbd2-402a-936a-cbb5694e6445',
  'type': 'tool_call'}]